## Exercise 12. Implement Logistic Regression with Mini-batch Gradient Descent using TensorFlow
Train it and evaluate it on the moons dataset (introduced in Chapter 5). Try
adding all the bells and whistles:<br/>
• Define the graph within a logistic_regression() function that can be reused
easily.<br/>
• Save checkpoints using a Saver at regular intervals during training, and save
the final model at the end of training.<br/>
• Restore the last checkpoint upon startup if training was interrupted.<br/>
• Define the graph using nice scopes so the graph looks good in TensorBoard.<br/>
• Add summaries to visualize the learning curves in TensorBoard.<br/>
• Try tweaking some hyperparameters such as the learning rate or the mini-
batch size and look at the shape of the learning curve.

### Setup

In [2]:
import mkl
mkl.get_max_threads()

2

In [3]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import numpy.random as rnd
import os

# to make this notebook's output stable across runs
rnd.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "tensorflow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [21]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [5]:
iris = load_iris()

In [8]:
X = iris.data
y = iris.target

In [9]:
m,n = X.data.shape
print('#Instances', m, '#Features', n)

#Instances 150 #Features 4


In [12]:
scaler = StandardScaler()

In [15]:
X_scaled = scaler.fit_transform(X)

In [17]:
X.sum(axis=0), X_scaled.sum(axis=0)

(array([ 876.5,  458.1,  563.8,  179.8]),
 array([ -2.53547183e-13,  -2.45553577e-13,  -2.22377672e-13,
         -2.43471909e-13]))

### Solution

Logistic Regression:

Prediction
p = sigma(W*x + b)
sigma(z) = 1 / (1 + exp(-z))

Loss
L = -(1/m) * sum(y * log(p) + (1-y) * log(1-p))
dL/dwj = (1/m) * sum(p - y) * xj

#### Base line solution (by Scikit-Learn)

In [18]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression()
reg.fit(X_scaled, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
reg.coef_, reg.intercept_

(array([[-0.81016631,  1.39369878, -1.68738578, -1.51899135],
        [ 0.13037985, -1.2463382 ,  0.78919477, -0.88943988],
        [ 0.01299039, -0.1445346 ,  1.86317337,  2.69887272]]),
 array([-1.61861686, -0.89769778, -2.70851029]))

#### TensorFlow

In [76]:
n_epochs = 100001
learning_rate = 0.001

X_op = tf.placeholder(dtype=np.float64, shape=(None, n), name='X')
y_op = tf.placeholder(dtype=np.float64, shape=(None, 1), name='y')
W = tf.Variable(initial_value=np.random.randn(n, 1), name='W')
b = tf.Variable(initial_value=np.zeros((1,1)), name='b')
p = tf.sigmoid(tf.matmul(X_op, W) + b, name='prediction')
# loss = tf.losses.log_loss(y_op, p, scope='loss')
loss = tf.reduce_mean(-tf.log(p) * y_op)
gradients = tf.gradients(loss, [W])[0]
train_op = tf.assign(W, W - gradients * learning_rate)

feed_dict = {X_op : X_scaled, y_op : y.reshape((-1, 1))}

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        sess.run(train_op, feed_dict=feed_dict)
        if epoch % 5000 == 0:
            print('Epoch', epoch, 'loss', sess.run(loss, feed_dict=feed_dict))
    
    print('Final weights:', sess.run(W))

Epoch 0 loss 0.364803930181
Epoch 5000 loss 0.17283139084
Epoch 10000 loss 0.123850291703
Epoch 15000 loss 0.099680961851
Epoch 20000 loss 0.0848750721397
Epoch 25000 loss 0.074726873854
Epoch 30000 loss 0.067267056679
Epoch 35000 loss 0.0615133565135
Epoch 40000 loss 0.0569167312832
Epoch 45000 loss 0.0531444755301
Epoch 50000 loss 0.049982380163
Epoch 55000 loss 0.0472858458247
Epoch 60000 loss 0.0449534778041
Epoch 65000 loss 0.0429119176552
Epoch 70000 loss 0.0411066821754
Epoch 75000 loss 0.0394963955113
Epoch 80000 loss 0.0380490276877
Epoch 85000 loss 0.0367393665913
Epoch 90000 loss 0.0355472739815
Epoch 95000 loss 0.0344564545427
Epoch 100000 loss 0.0334535693248
Final weights: [[ 0.75111156]
 [-2.64949059]
 [ 3.29147732]
 [ 1.86372929]]
